In [8]:
# Connect to your SpatiaLite database
import sqlite3
import os
import pandas as pd
from shapely.geometry import LineString
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
import geopandas as gpd
from shapely import wkt
import folium

from shapely.geometry import MultiPolygon, Polygon
from shapely.wkt import loads
import numpy as np
import re

from shapely.geometry import MultiPolygon, Polygon
from shapely.wkt import loads


spatialite_path = "C:/Users/user/Documents/mod_spatialite-5.1.0-win-amd64"
os.environ["PATH"] = spatialite_path + ";" + os.environ["PATH"]

# create a new sqlite data base
conn = sqlite3.connect("testing.sqlite")
conn.enable_load_extension(True)
conn.load_extension("mod_spatialite")

# mod_spatialite (recommended)
conn.execute('SELECT load_extension("mod_spatialite")')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# initiate a spatial metadata for the sqlite database 
cursor.execute("SELECT InitSpatialMetadata(1);")

# Checking spatialite version 
res = cursor.execute("SELECT spatialite_version()")
print(res.fetchone())

# Checking Sqlite Version
res1 = cursor.execute("SELECT sqlite_version()")
print(res1.fetchone())


('5.1.0',)
('3.45.1',)


In [2]:
# Function to calculate the angle between three points
def calculate_angle(p1, p2, p3):
    # Create vectors
    v1 = (p1[0] - p2[0], p1[1] - p2[1])
    v2 = (p3[0] - p2[0], p3[1] - p2[1])
    # Calculate dot product and magnitudes
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)

    # Check if magnitudes are zero to avoid division by zero
    if magnitude_v1 == 0 or magnitude_v2 == 0:
        return np.nan  # Return NaN or some other value indicating undefined angle

    # Calculate the angle
    cos_angle = dot_product / (magnitude_v1 * magnitude_v2)
    # Avoid numerical issues due to floating-point arithmetic
    cos_angle = np.clip(cos_angle, -1.0, 1.0)
    angle = np.arccos(cos_angle)
    return np.degrees(angle)


def parse_wkt(wkt_polygon):

    # Regular expression to extract coordinates from WKT polygon
    coord_pattern = re.compile(r'(\d+(\.\d+)?)\s+(\d+(\.\d+)?)')
    
    # Find all matches of the coordinate pattern in the WKT polygon
    matches = coord_pattern.findall(wkt_polygon)
    
    # Convert each match to a tuple of numerical values (x, y)
    points = [(float(x), float(y)) for x, _, y, _ in matches]
    
    return points

# Assuming you have a table "polygons" with a geometry column "geom"
cur.execute("SELECT AsText(geom) FROM test_data WHERE id = ?", (2,))  # Adjust the ID as needed

# Fetch the WKT representation of the polygon
wkt = cur.fetchone()[0]

# # Assuming you have a function to parse WKT and extract points
# # This needs to be implemented based on your specific WKT format
points = parse_wkt(wkt)  # Implement this function to extract points from WKT
print(points)
    
pseudo_node_count = 0
# Adjust the angle threshold to detect angles significantly less than 180 degrees
angle_threshold = 10  # Degrees

# Initialize a list to store the indices of pseudo nodes
pseudo_nodes = []

# Calculate angles and identify pseudo nodes
for i in range(len(points)):
    p1 = points[i - 1]  # Previous point
    p2 = points[i]      # Current point
    p3 = points[(i + 1) % len(points)]  # Next point, wrapping around using modulo

    angle = calculate_angle(p1, p2, p3)
    # Look for angles significantly less than 180 degrees
    print(angle)
    if angle < angle_threshold or angle == 180:
        pseudo_nodes.append(i)

# The count of pseudo nodes is the length of the pseudo_nodes list
pseudo_node_count = len(pseudo_nodes)

print("Indices of pseudo nodes:", pseudo_nodes)
print("Number of pseudo nodes:", pseudo_node_count)
# Close the database connection
# conn.close()


NameError: name 'cur' is not defined

In [9]:
# Function to calculate the angle between three points
def calculate_angle(p1, p2, p3):
    # Create vectors
    v1 = (p1[0] - p2[0], p1[1] - p2[1])
    v2 = (p3[0] - p2[0], p3[1] - p2[1])
    # Calculate dot product and magnitudes
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)

    # Check if magnitudes are zero to avoid division by zero
    if magnitude_v1 == 0 or magnitude_v2 == 0:
        return np.nan  # Return NaN or some other value indicating undefined angle

    # Calculate the angle
    cos_angle = dot_product / (magnitude_v1 * magnitude_v2)
    # Avoid numerical issues due to floating-point arithmetic
    cos_angle = np.clip(cos_angle, -1.0, 1.0)
    angle = np.arccos(cos_angle)
    return np.degrees(angle)


def parse_wkt(wkt_polygon):

    # Regular expression to extract coordinates from WKT polygon
    coord_pattern = re.compile(r'(\d+(\.\d+)?)\s+(\d+(\.\d+)?)')
    
    # Find all matches of the coordinate pattern in the WKT polygon
    matches = coord_pattern.findall(wkt_polygon)
    
    # Convert each match to a tuple of numerical values (x, y)
    points = [(float(x), float(y)) for x, _, y, _ in matches]
    
    return points

def pseudo_node(geotypes):
    selected_data = customFetchAll(f"select id, AsText(geom) from test data where geotypes='{geotypes}'")
    # Assuming you have a table "polygons" with a geometry column "geom"
    
    # Fetch the WKT representation of the polygon
    for wkt in selected_data: 
        # # Assuming you have a function to parse WKT and extract points
        # # This needs to be implemented based on your specific WKT format
        points = parse_wkt(wkt[1])  # Implement this function to extract points from WKT

        pseudo_node_count = 0
        # Adjust the angle threshold to detect angles significantly less than 180 degrees
        angle_threshold = 10  # Degrees

        # Initialize a list to store the indices of pseudo nodes
        pseudo_nodes = []
        polygon_pseudo = []

        # Calculate angles and identify pseudo nodes
        for i in range(len(points)):
            p1 = points[i - 1]  # Previous point
            p2 = points[i]      # Current point
            p3 = points[(i + 1) % len(points)]  # Next point, wrapping around using modulo

            angle = calculate_angle(p1, p2, p3)
            # Look for angles significantly less than 180 degrees
            print(angle)
            if angle < angle_threshold or angle == 180:
                pseudo_nodes.append(i)
                polygon_pseudo.append(wkt)

        # The count of pseudo nodes is the length of the pseudo_nodes list
        pseudo_node_count = len(pseudo_nodes)

        print("Indices of pseudo nodes:", pseudo_nodes)
        print("Number of pseudo nodes:", pseudo_node_count)


In [16]:
# create a new sqlite data base
# conn = sqlite3.connect("testing.sqlite")
# conn.enable_load_extension(True)
# conn.load_extension("mod_spatialite")

# # mod_spatialite (recommended)
# conn.execute('SELECT load_extension("mod_spatialite")')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# initiate a spatial metadata for the sqlite database 
cursor.execute("SELECT InitSpatialMetadata(1);")


# cursor.execute("CREATE TABLE new_data (id INTEGER PRIMARY KEY, geotypes VARCHAR,  geom BLOB);")
# cursor.execute("INSERT INTO new_data (geotypes, geom) VALUES ('Polygon', ST_GeomFromText('POLYGON((0 0, 5 0, 5 5, 0 5, 0 0))',4326));")
# cursor.execute("INSERT INTO new_data (geotypes, geom) VALUES ('Polygon', ST_GeomFromText('POLYGON((20 20, 25 20, 25 25, 20 25, 20 20))',4326));")
# cursor.execute("INSERT INTO test_data (geotypes, geom) VALUES ('Polygon', ST_GeomFromText('POINT(0 10)',4326));")

# cursor.execute("UPDATE test_data SET geotypes = 'MultiPolygon', geom = ST_GeomFromText('POINT(10 10)',4326)  WHERE id= 7;")


# conn.commit()


In [21]:
def customQuery(Query):
    database = sqlite3.connect('testing.sqlite')
    dataframe = pd.read_sql_query(Query, database)
    print(dataframe)
    return dataframe

def customFetchOne(Fetch):
#     cursor.execute("select ST_Touches(r1.geom, r2.geom) as check1 from spatial_data r1 cross join spatial_data r2 where r1.id = 4 and r2.id = 2;")
    cursor.execute(Fetch)
    res = cursor.fetchone()
#     print(res)
    return res[0]

def customFetchAll(Fetch):
#     cursor.execute("select ST_Touches(r1.geom, r2.geom) as check1 from spatial_data r1 cross join spatial_data r2 where r1.id = 4 and r2.id = 2;")
    cursor.execute(Fetch)
    res = cursor.fetchall()
#     print(res)
    return res

# customQuery("select * from test_data")


# print(customFetchOne("delete from test_data where id=5"))

def cleanPseudoNode(geotypes):
    selecteddata = customFetchAll(f"select id from test_data where geotypes= '{geotypes}'")
    for i in selecteddata:
        fix_data = customFetchOne(f"SELECT AsText(ST_SimplifyPreserveTopology(geom, 0.5)) FROM test_data where id={i[0]}")
#         print(f"UPDATE test_data SET geom = ST_GeomFromText('{fix_data}',4326) where id={i[0]};")
#         cursor.execute(f"UPDATE test_data SET geom = ST_GeomFromText('{fix_data}',4326) where id={i[0]};")
        print(fix_data)
    print("all data which contain pseudo node has been fixed")

    
cleanPseudoNode("Polygon")

def checkCluster():
    cluster_data = customFetchAll("SELECT a.id AS point_id, b.id AS cluster_id, ST_Distance(a.geom, b.geom) AS distance FROM test_data AS a, test_data AS b WHERE a.id != b.id AND ST_Distance(a.geom, b.geom) < 1 AND ST_Distance(a.geom, b.geom) > 0.1")
    # Create a set to store unique tuples
    unique_tuples = set()

    # Iterate over the input data
    for tuple_value in cluster_data:
        # Check if the tuple meets the condition
        if (tuple_value[0], tuple_value[1], tuple_value[2]) not in unique_tuples and
           (tuple_value[1], tuple_value[0], tuple_value[2]) not in unique_tuples:
            # Add the tuple to the set
            unique_tuples.add((tuple_value[0], tuple_value[1], tuple_value[2]))

    # Convert the set back to a list
    unique_data = list(unique_tuples)
    
    return unique_data

#     print("Input data:", cluster_data)
    print("Output data:", unique_data)
    
    
#     return cluster_data

# checkCluster()
# print(customFetchAll("SELECT a.id AS point_id, b.id AS cluster_id, ST_Distance(a.geom, b.geom) AS distance FROM test_data AS a, test_data AS b WHERE a.id != b.id AND ST_Distance(a.geom, b.geom) < 1"))

# UPDATE test_data SET geom = ST_GeomFromText('POINT(10 10)',4326)  WHERE id= 7;

# print(customFetchOne("SELECT AsText(ST_SimplifyPreserveTopology(geom, 0.5)) FROM test_data where id=4"))
# print(customFetchOne("SELECT ST_AsText(geom) from test_data where id =7;"))
# cursor.execute("delete from test_data where id=3;")
# conn.commit()

update test_data set geom = ST_GeomFromText('POLYGON((20 20, 30 20, 30 10, 20 10, 20 20))',4326) where id=1;
POLYGON((20 20, 30 20, 30 10, 20 10, 20 20))
update test_data set geom = ST_GeomFromText('POLYGON((5 0, 10 10, 0 10, 5 0))',4326) where id=3;
POLYGON((5 0, 10 10, 0 10, 5 0))
update test_data set geom = ST_GeomFromText('POLYGON((10 5, 10 20, 40 20, 40 5, 10 5))',4326) where id=4;
POLYGON((10 5, 10 20, 40 20, 40 5, 10 5))
update test_data set geom = ST_GeomFromText('POLYGON((10 5, 10 20, 40 20, 40 5, 10 5))',4326) where id=5;
POLYGON((10 5, 10 20, 40 20, 40 5, 10 5))
update test_data set geom = ST_GeomFromText('POLYGON((15 20, 25 20, 25 10, 15 10, 15 20))',4326) where id=6;
POLYGON((15 20, 25 20, 25 10, 15 10, 15 20))
all data which contain pseudo node has been fixed


In [163]:
data = customFetchAll("select id, geotypes, ST_AsText(geom) as geometry from test_data where geotypes='Polygon'")
print(data)

def wkt2map(data):
    # Fix WKT strings to ensure polygons are properly closed
    data = [(id, typ, wkt_string.rstrip(')') + ', {})'.format(wkt_string.split('((')[-1])) for id, typ, wkt_string in data]
    # Parse WKT strings and create Shapely geometry objects
    geometries = [wkt.loads(wkt_string) for _, _, wkt_string in data]
    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(data, columns=['ID', 'Type', 'WKT'], geometry=geometries, crs='EPSG:4326')
    # Create a Folium map centered at a specific location
    m = folium.Map(location=[15, 20], zoom_start=4)
    # Add GeoDataFrame to the map
    folium.GeoJson(gdf).add_to(m)
    # Display the map
    return m

wkt2map(data)



[(1, 'Polygon', 'POLYGON((20 20, 30 20, 30 10, 20 10, 20 20))'), (2, 'Polygon', 'POLYGON((0 0, 3 0, 3 1, 3.5 1, 3 1, 3 3, 0 3, 0 0))'), (3, 'Polygon', 'POLYGON((5 0, 10 10, 0 10, 5 0))'), (4, 'Polygon', 'POLYGON((10 5, 10 10, 10 20, 40 20, 40 15, 40 5, 20 5, 10 5))'), (5, 'Polygon', 'POLYGON((10 5, 10 20, 40 20, 40 5, 10 5))'), (6, 'Polygon', 'POLYGON((15 20, 25 20, 25 10, 15 10, 15 20))')]
